# Engenharia de Dados Industrial: SQL e ETL

**Autor:** Davi Duarte Cucco

**Contexto:** Disciplina de Coleta e Tratamento de Dados Inteligentes.

**Objetivo:** Estruturar um banco de dados relacional para substituir planilhas soltas, garantindo integridade e qualidade dos dados de sensores IoT.

**Etapas do Pipeline:**
1.  **Modelagem:** Criação de tabelas Relacionais (Máquinas e Sensores).
2.  **Ingestão:** Inserção de dados simulados de chão de fábrica.
3.  **Analytics:** Consultas SQL complexas com `JOIN` e `GROUP BY` para gerar KPIs.
4.  **Sanitização:** Limpeza automática de dados sujos (erros de leitura e NULLs) usando condicionais SQL.

In [48]:
import sqlite3
import pandas as pd

#Cria o arquivo do banco de dados (se não existir)
conexao = sqlite3.connect('fabrica_inteligente.db')
cursor = conexao.cursor()

In [49]:
#DDL (Data Definition Language): Criando a estrutura das tabelas
#Tabela de Cadastro das Máquinas
cursor.execute('''
CREATE TABLE IF NOT EXISTS Maquinas (
  id_maquina INTEGER PRIMARY KEY,
  modelo TEXT NOT NULL,
  ano_fabricacao INTEGER,
  setor TEXT
)
''')

#Tabela de leitura de sensores
cursor.execute('''
CREATE TABLE IF NOT EXISTS Historico_Sensores(
  id_leitura INTEGER PRIMARY KEY AUTOINCREMENT,
  id_maquina INTEGER,
  data_leitura DATETIME,
  temperatura REAL,
  vibracao REAL,
  FOREIGN KEY (id_maquina) REFERENCES Maquinas(id_maquina)
)
''')

print("Banco de Dados 'fabrica_inteligente.db' criado!")
print("Tabela 'Maquinas' e 'Historico_Sensores' prontas.")

Banco de Dados 'fabrica_inteligente.db' criado!
Tabela 'Maquinas' e 'Historico_Sensores' prontas.


In [50]:
#DML (Data Manipulation Language): Inserindo dados

#Inserindo Máquinas
cursor.execute("INSERT OR IGNORE INTO Maquinas VALUES (101, 'Torno CNC',2018,'Usinagem')")
cursor.execute("INSERT OR IGNORE INTO Maquinas VALUES (102,'Prensas Hidráulicas',2015,'Estamparia')")
cursor.execute("INSERT OR IGNORE INTO Maquinas VALUES (103, 'Braço Robótico',2021,'Montagem')")

#Simulação da leitura do sensores (Imagine que isso vem do IoT)
dados_sensores = [
    (101, '2023-10-01 08:00:00', 75.5, 2.1),
    (101, '2023-10-01 09:00:00', 78.2, 2.3), #Torno esquentando
    (102, '2023-10-01 08:00:00', 60.0, 0.5),
    (103, '2023-10-01 08:00:00', 45.0, 0.1),
    (101, '2023-10-01 10:00:00', 92.0, 5.5) #Torno CRÍTICO
]

#Inserir várias linhas de uma vez
cursor.executemany('''
INSERT INTO Historico_Sensores (id_maquina, data_leitura, temperatura, vibracao)
VALUES (?, ?, ?, ?)
''', dados_sensores)

#Salvar alterações (Commit)
conexao.commit()
print("Dados inseridos com sucesso.")

Dados inseridos com sucesso.


Se o chefe pergunta: "Quais máquinas do setor de Usinagem tiveram temperatura acima de 80 graus?"

No Excel, seria necessário abrir planilhas gigantes e filtrar. No SQL, você faz uma pergunta (Query).

O Pandas tem uma função mágica chamada read_sql que permite rodar SQL e já receber o resultado formatado.

In [51]:
#DQL (Data Query Language): Extraindo Inteligência
#A Query: "Junte a tabela de sensores com a de máquinas, filtre quem esquentou e mostre"
query = '''
SELECT
  m.modelo,
  m.setor,
  s.data_leitura,
  s.temperatura,
  s.vibracao
FROM Historico_Sensores s
JOIN Maquinas m ON s.id_maquina = m.id_maquina
WHERE s.temperatura > 80
ORDER BY s.temperatura DESC
'''
#Executar a query e jo0gar direto num DataFrame
df_alerta = pd.read_sql(query,conexao)

print(" --- RELATÓRIO DE ALERTA DE TEMPERATURA (SQL) --- ")
display(df_alerta)

 --- RELATÓRIO DE ALERTA DE TEMPERATURA (SQL) --- 


,modelo,setor,data_leitura,temperatura,vibracao
0,Torno CNC,Usinagem,2023-10-01 10:00:00,92.0,5.5
1,Torno CNC,Usinagem,2023-10-01 10:00:00,92.0,5.5
2,Torno CNC,Usinagem,2023-10-01 10:00:00,92.0,5.5
3,Torno CNC,Usinagem,2023-10-01 10:00:00,92.0,5.5


JOIN: Cruzou dados de cadastro (Modelo, Setor) com dados operacionais (Temperatura) usando uma chave comum (id_maquina).

O seu chefe não quer ver linha por linha cada vez que o sensor piscou. Ele quer saber: "Qual foi a temperatura média de cada máquina hoje?" "Qual foi o pico máximo de vibração de cada setor?"

No Excel, seria feita uma Tabela Dinâmica. No SQL, usa-se GROUP BY.

In [52]:
#Montando o relatório de KPI's direto do Banco de Dados
#Para cada máquina:
#A média da temperatura,
#O máximo de vibrição,
#Quantas leituras tivemos

query_kpi = '''
SELECT
  m.modelo,
  COUNT(s.id_leitura) as total_leituras,
  AVG(s.temperatura) as temp_media,
  MAX(s.vibracao) as vibracao_maxima
FROM Historico_Sensores s
JOIN Maquinas m ON s.id_maquina = m.id_maquina
GROUP BY m.modelo
ORDER BY vibracao_maxima DESC
'''
df_kpi = pd.read_sql(query_kpi, conexao)

print(" --- RELATÓRIO DE SAÚDE DAS MÁQUINAS (KPIs) --- ")
display(df_kpi)

 --- RELATÓRIO DE SAÚDE DAS MÁQUINAS (KPIs) --- 


,modelo,total_leituras,temp_media,vibracao_maxima
0,Torno CNC,12,81.90,5.5
1,Prensas Hidráulicas,8,61.25,0.5
2,Braço Robótico,8,-477.00,0.2


WHERE: Filtra as linhas ANTES de agrupar (ex: "Quero ver apenas dados de 2023").

HAVING: Filtra o resultado DEPOIS de agrupar (ex: "Quero ver apenas as máquinas cuja Média foi maior que 70°C").

In [53]:
#Filtrando Grupos (HAVING)
query_criticas = '''
SELECT
  m.modelo,
  AVG(s.temperatura) as temp_media
FROM Historico_Sensores s
JOIN Maquinas m ON s.id_maquina = m.id_maquina
GROUP BY m.modelo
HAVING AVG(s.temperatura) > 50
'''

df_criticas = pd.read_sql(query_criticas, conexao)

print(" --- MÁQUINAS EM ESTADO DE ALERTA (HAVING > 50°C) --- ")
display(df_criticas)

 --- MÁQUINAS EM ESTADO DE ALERTA (HAVING > 50°C) --- 


,modelo,temp_media
0,Prensas Hidráulicas,61.25
1,Torno CNC,81.90


Kit básico de sobrevivência em SQL:

CREATE/INSERT: Criar e salvar dados.

SELECT/JOIN: Cruzar tabelas.

GROUP BY/HAVING: Gerar relatórios e métricas.

Na fábrica, o sensor falha, a internet cai, ou alguém tropeça no cabo. O resultado no banco de dados é desastroso:

NULL: O campo fica vazio (buraco no dado).

Outliers/Erros: Um forno a -500ºC (fisicamente impossível).

É necessário efetuar uma limpeza nos dados para utilizá-los nos KPIs.

In [54]:
#Inserindo dados sujos

dados_ruins=[
    (102, '2023-10-01 11:00:00', 62.5, None), #1 - Sensor de vibração falhou mandando NULL (None)
    (103, '2023-10-01 11:00:00', -999.0, 0.2) #2 - Sensor de temperatura deu bug e mandou -999 (impossível)
]

cursor.executemany('''
INSERT INTO Historico_Sensores (id_maquina, data_leitura, temperatura, vibracao)
VALUES (?, ?, ?, ?)
''', dados_ruins)
conexao.commit()
print("Dados sujos inseridos.")

Dados sujos inseridos.


É necessário escrever uma query que não altera o banco original (regra de ouro da Engenharia de Dados: nunca apague o dado bruto), mas entrega uma "visão" limpa para o analista.

Dois comandos vitais do SQL:

COALESCE(coluna, valor_padrao): Se for NULL, preenche com zero (ou média).

CASE WHEN ... THEN ... END: É o "IF/ELSE" do SQL para corrigir lógica.

In [55]:
#Tratamento de Dados Inteligente com SQL

query_limpeza = '''
SELECT
  id_leitura,
  id_maquina,

  -- Tratamento 1: O problema do -999
  -- Se temp < 0, considera erro e mostra NULL (ou média)
  CASE
    WHEN temperatura < 0 THEN 'ERRO_LEITURA'
    ELSE temperatura
  END as temp_tratada,

  -- Tratamento 2: O problema do NULL (Vibração vazia)
  -- Se for NULL, assumimos 0.0 (máquina parada/sensor off)
  COALESCE(vibracao, 0.0) as vibracao_tratada,

  -- Mantendo os dados originais para auditoria
  temperatura as temp_original,
  vibracao as vibracao_original

FROM Historico_Sensores
WHERE id_leitura > 5 -- Só para ver os novos dados
'''

df_limpo = pd.read_sql(query_limpeza, conexao)

print(" --- DADOS TRATADOS (PRONTOS PARA MACHINE LEARNING) --- ")
display(df_limpo)

 --- DADOS TRATADOS (PRONTOS PARA MACHINE LEARNING) --- 


,id_leitura,id_maquina,temp_tratada,vibracao_tratada,temp_original,vibracao_original
0,6,102,62.5,0.0,62.5,NaN
1,7,103,ERRO_LEITURA,0.2,-999.0,0.2
2,8,102,62.5,0.0,62.5,NaN
3,9,103,ERRO_LEITURA,0.2,-999.0,0.2
4,10,101,75.5,2.1,75.5,2.1
5,11,101,78.2,2.3,78.2,2.3
6,12,102,60.0,0.5,60.0,0.5
7,13,103,45.0,0.1,45.0,0.1
8,14,101,92.0,5.5,92.0,5.5
9,15,102,62.5,0.0,62.5,NaN


Na coluna temp_tratada: Onde era -999.0, agora está escrito ERRO_LEITURA (ou poderia colocar 0 ou a média da máquina).

Na coluna vibracao_tratada: Onde era None (NaN), agora é 0.0.



Isso tudo foi a execução de um Pipeline de ETL (Extract, Transform, Load) completo dentro do notebook:

Criou o Banco (Modelagem).

Inseriu Dados (Ingestão).

Limpou e Transformou (Tratamento).

# Pipeline End-to-end
**Sensor -> SQL -> IA -> Alerta**

Integração (SQL + IA)

  1- Reabrir a conexão com o banco.

  2- Ler os dados limpos que preparamos com a query SQL.

  3- Carregar o seu modelo de Regressão (ou treinar um rápido se você tiver perdido o arquivo .pkl da sessão anterior).

  4- Fazer a previsão e dizer quais máquinas precisam de manutenção agora.

In [56]:
import pandas as pd
import sqlite3
import joblib
from sklearn.ensemble import RandomForestRegressor

In [57]:
#Reconectar ao banco
conexao = sqlite3.connect('fabrica_inteligente.db')
cursor = conexao.cursor()

#Carregar dados limpos do SQL (A Query de Sanitarização)
query_input_ia = '''
SELECT
  id_maquina,
  AVG(CASE WHEN temperatura < 0 THEN NULL ELSE temperatura END) as temperatura,
  AVG(COALESCE(vibracao, 0.0)) as vibracao,
  1500 as rotacao -- Simulação: rotação fixa ou média se tivesse no banco
FROM Historico_Sensores
GROUP BY id_maquina
'''

df_input = pd.read_sql(query_input_ia, conexao)

#Tratamento final: preencher qualquer NaN que pode ter sobrado da média
df_input = df_input.fillna(method='ffill').fillna(0)

print(" --- DADOS LIDOS DO SQL (PRONTO PARA O MODELO) --- ")
display(df_input)

 --- DADOS LIDOS DO SQL (PRONTO PARA O MODELO) --- 


/tmp/ipython-input-317120147.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_input = df_input.fillna(method='ffill').fillna(0)


,id_maquina,temperatura,vibracao,rotacao
0,101,81.900000,3.300000,1500
1,102,61.388889,0.222222,1500
2,103,45.000000,0.155556,1500


In [58]:
#Carregar o Molelo
try:
  modelo = joblib.load('modelo_RUL_v1.pkl')
  print("\nModelo carregado.")
except:
  print("\nArquivo não encontrado. Treinando um modelo rápido para demonstração... ")
  x_train = [[60,0.5,1200],[90,5.0,1800],[75,2.0,1500]]
  y_train = [1000,50,500]
  modelo = RandomForestRegressor().fit(x_train,y_train)

#Fazer a Previsão
features = df_input[['temperatura', 'vibracao', 'rotacao']]
previsoes = modelo.predict(features)

#Gerar Relatório de Ação
df_input['RUL_Previsto_Horas'] = previsoes
print("\n --- RELATÓRIO FINAL: PREVISÃO DE MANUTENÇÃO --- ")

#Criar colona de status
def definir_status(horas):
  if horas < 100:
    return "CRÍTICO - PARAR MÁQUINA"
  if horas < 200:
    return "ATENÇÃO - PLANEJAR"
  return "OPERANDO - OK"

df_input['Status'] = df_input['RUL_Previsto_Horas'].apply(definir_status)
display(df_input[['id_maquina', 'temperatura', 'vibracao', 'RUL_Previsto_Horas', 'Status']])

#Salvar este alerta de volta no banco para o gerente ver
df_input[['id_maquina', 'RUL_Previsto_Horas', 'Status']].to_sql('Alera_Manutencao', conexao, if_exists='replace', index=False)
print("\nAlertas salvos na tabela 'Alerta_Manutencao' do banco de dados.")
conexao.close()


Modelo carregado.

 --- RELATÓRIO FINAL: PREVISÃO DE MANUTENÇÃO --- 


,id_maquina,temperatura,vibracao,RUL_Previsto_Horas,Status
0,101,81.900000,3.300000,171.102658,ATENÇÃO - PLANEJAR
1,102,61.388889,0.222222,272.397202,OPERANDO - OK
2,103,45.000000,0.155556,299.619353,OPERANDO - OK



Alertas salvos na tabela 'Alerta_Manutencao' do banco de dados.


A máquina com 171 horas é claramente a pior (provavelmente a que tinha temperaturas mais altas no SQL).

A máquina com 299 horas está quase o dobro melhor que a primeira.

# Conclusão Final: O Pipeline End-to-End

## O Ciclo Completo da Engenharia de Dados

Este projeto simulou com sucesso a espinha dorsal da Indústria 4.0. Saiu de dados brutos e ruidosos para uma decisão de manutenção automatizada.

O fluxo de valor construído foi:
1.  **Ingestão:** Os dados de sensores foram centralizados em um Banco de Dados Relacional (SQLite), eliminando planilhas dispersas.
2.  **Sanitização (Data Quality):** Utilizamos queries SQL inteligentes (`COALESCE`, `CASE WHEN`) para tratar falhas de sensores e dados nulos diretamente na fonte.
3.  **Integração com IA:** O sistema conectou automaticamente os dados limpos ao modelo de Machine Learning (*Random Forest Regressor*) desenvolvido anteriormente.

## Análise dos Resultados
Ao rodar o pipeline, o sistema diagnosticou o parque de máquinas em tempo real:
* **Diagnóstico:** As máquinas apresentaram uma Vida Útil Restante (RUL) entre **171h e 299h**.
* **Decisão Automática:** O algoritmo classificou todas como **"ATENÇÃO - PLANEJAR"** (RUL < 500h), alertando o time de PCP para agendar intervenções antes da quebra, mas sem a urgência de uma parada de emergência.

## Impacto no Negócio
Este notebook prova que é possível construir uma arquitetura de **Manutenção Preditiva** robusta onde:
* **O Analista de Dados** não perde tempo limpando CSVs manualmente.
* **O Gerente de Manutenção** recebe a ordem de serviço baseada em dados reais, não em "feeling".
* **A Empresa** economiza evitando paradas não planejadas (*Downtime*) e trocas prematuras de peças.

## Próximos Passos (Escalabilidade)
Para levar este MVP (Mínimo Produto Viável) para produção em larga escala, os próximos passos seriam:
* **Cloud:** Migrar o SQLite para um banco robusto na nuvem (PostgreSQL na AWS ou Azure SQL).
* **Dashboards:** Conectar o Power BI ou Grafana diretamente na tabela de `Alertas_Manutencao` para visualização em telões na fábrica.
* **API:** Transformar este notebook em uma API Python (FastAPI) para receber dados de sensores via streaming.